In [1]:
import redis
import tushare as ts
import psycopg2
import datetime
import pandas as pd
import numpy as np
import easyquotation
import easytrader
import re

In [2]:
pro = ts.pro_api('ebd374898a15c69a938b0b6c2de6beb2854b694a4bbe6c731640bfc4')

conn = psycopg2.connect(database='stock', user='postgres',password='csubigdata',host='localhost',port='5432')
cur = conn.cursor()

pool = redis.ConnectionPool(host='127.0.0.1', port=6379)
r = redis.Redis(connection_pool=pool, decode_responses=True)


In [29]:
quotation = easyquotation.use('sina')
data = quotation.market_snapshot(prefix=True)
sql = 'select id,code from stockorder'
cur.execute(sql)
dbData = cur.fetchall()
dbData = pd.DataFrame(dbData,columns=['id','code'])
for i in range(len(dbData)):
    # 依次查找每一个股票的实时价格数据
    code = dbData.loc[i,'code'].split('.')[0]
    for k,v in data.items():
        if code in k:
            sqlUpdate = 'update stockorder set price = %f where id =%d'%(v['now'],dbData.loc[i,'id'])
            cur.execute(sqlUpdate)
            conn.commit()

姚记科技
26.22
福莱特
29.6
南京新百
8.03
华明装备
12.09


In [1]:
from PIL import Image
import pytesseract
img_path = "C:\\Users\\lihan\\Desktop\\teste\\tmp.png"
im = Image.open(img_path).convert("L")
# 1. threshold the image
threshold = 200
table = []
for i in range(256):
    if i < threshold:
        table.append(0)
    else:
        table.append(1)
out = im.point(table, "1")
num = pytesseract.image_to_string(out)
print(num)

7796


In [20]:
#将股票代码转换为easyquotation开源接口格式
def denormalize_code(code:str):
    dic_code = {'XSHE':'sz','XSHG':'sh'}
    code_orig = code[7:]
    code_new = dic_code[code_orig] + code[:6]
    return code_new

In [15]:
day = '2023-09-14'
sql = 'select id from stockorder where created_time =\'%s\''%(day)
cur.execute(sql)
data = cur.fetchall()
data
for e in data:
    print(e[0])
    if e[0] == 25:
        sql = 'delete from stockorder where id = %d'%(e[0])
        print(sql)
        cur.execute(sql)
        conn.commit()

23
24
25
delete from stockorder where id = 25


In [1]:
#设置一个全局变量来控制股票订单
update_dict = {}

In [25]:
quotation = easyquotation.use('sina')
bigdata1 = easytrader.use('universal_client')#定义使用universal_client
bigdata1.prepare(user='123456789', comm_password='123456',exe_path=r'C:\\ths\\xiadan.exe')

#将股票代码转换为easyquotation开源接口格式
def denormalize_code(code:str):
    dic_code = {'XSHE':'sz','XSHG':'sh'}
    code_orig = code[7:]
    code_new = dic_code[code_orig] + code[:6]
    return code_new


c:\Users\lihan\.conda\envs\bigdata\lib\site-packages\pywinauto\application.py:1064: UserWarning: 32-bit application should be automated using 32-bit Python (you use 64-bit Python)
  UserWarning)


In [26]:
# 获取观望中/已买入所有订单
sql = 'select id,code,state,updated_time,stra_param_buy,stra_param_sell,amount from stockorder where state = 0 or state = 1'
cur.execute(sql)
order = cur.fetchall()
order_df = pd.DataFrame(order, columns=['id','code','state','updated_time','stra_param_buy','stra_param_sell','amount'])
# 存储操作记录用于更新数据库
global update_dict 
# 获取实时数据
data = quotation.market_snapshot(prefix=True)

for index,row in order_df.iterrows():
    code = row['code']
    id = row['id']
    
    amount = row['amount']
    code_denomalized = denormalize_code(code)
    # 如果传入数据没有这个股票的实时信息，则略过
    if code_denomalized not in data.keys():
        continue
    else:
        # 捕获当前的价格
        now = data[code_denomalized]['now']
        print(code,now)
        # 如果状态为未买入
        if row['state'] == 0:
            stra_buy = re.split(r"[ ,]",row['stra_param_buy'])
            if stra_buy[0] == 'price:':
                # 如果是price策略，则判断
                target_price = float(stra_buy[1])
                if now <= target_price:
                    # 下单会返回一个委托单号{'entrust_no':},如果成交/错误则返回{'message':}
                    return_info = bigdata1.buy(code,amount,target_price)
                    # 已经存在下单单号
                    if 'entrust_no' in return_info.keys():
                        update_dict[return_info['entrust_no']] = id
            elif stra_buy[0] == 'percent:':
                # 如果是percent策略，则判断买入价格是否到达对应百分比，默认买入是1-percent
                target_percent = float(stra_buy[1])
                target_price = float(stra_buy[3])
                target_price = target_price*(1-0.01*target_percent)
                if now <= target_price:
                    # 下单会返回一个委托单号{'entrust_no':},如果成交/错误则返回{'message':}
                    return_info = bigdata1.buy(code,amount,target_price)
                    # 已经存在下单单号
                    if 'entrust_no' in return_info.keys():
                        update_dict[return_info['entrust_no']] = id
            elif stra_buy[0] == 'ma1:':
                # 如果是MA策略，则判断对应股票是否在redis中
                current_day = datetime.date.today()
                DateKey = current_day.strftime("%Y%m%d")+":"+str(md1)+str(md2)
                result = r.get(DateKey)
                if result == None:
                    md1 = int(stra_buy[1])
                    md2 = int(stra_buy[3])
                    result = get_ma(md1=md1,md2=md2)

                result = json.loads(result)
                result = pd.DataFrame(result,columns=['name','code','close'])
                # 返回的dataframe中的全都是md1>md2的，则如果股票代码在其中就可以买入
                if not result[result['code'].str.contains(code)].empty:
                    return_info = bigdata1.buy(code,amount,now)
                    # 已经存在下单单号
                    if 'entrust_no' in return_info.keys():
                        update_dict[return_info['entrust_no']] = id

        if row['state'] == 1:
            # 如果状态为已买入,需要检查一下是否当天买入
            if row['updated_time'] == datetime.datetime.today:
                continue
            stra_sell = re.split(r"[ ,]",row['stra_param_sell'])
            if stra_sell[0] == 'price:':
                # 如果是price策略，则判断
                target_price = float(stra_sell[1])
                if now >= target_price:
                    # 下单会返回一个委托单号{'entrust_no':},如果成交/错误则返回{'message':}
                    return_info = bigdata1.sell(code,amount,target_price)
                    # 已经存在下单单号
                    if 'entrust_no' in return_info.keys():
                        update_dict[return_info['entrust_no']] = id
            elif stra_sell[0] == 'percent:':
                # 如果是percent策略，则判断
                target_percent = float(stra_sell[1])
                target_price = float(stra_sell[3])
                target_price = target_price*(1+0.01*target_percent)
                if now >= target_price:
                    # 下单会返回一个委托单号{'entrust_no':},如果成交/错误则返回{'message':}
                    return_info = bigdata1.sell(code,amount,target_price)
                    # 已经存在下单单号
                    if 'entrust_no' in return_info.keys():
                        update_dict[return_info['entrust_no']] = id
            elif stra_sell[0] == 'ma1:':
                # 如果是MA策略，则判断
                # 如果是MA策略，则判断对应股票是否在redis中
                current_day = datetime.date.today()
                DateKey = current_day.strftime("%Y%m%d")+":"+str(md1)+str(md2)
                result = r.get(DateKey)
                if result == None:
                    md1 = int(stra_sell[1])
                    md2 = int(stra_sell[3])
                    result = get_ma(md1=md1,md2=md2)

                result = json.loads(result)
                result = pd.DataFrame(result,columns=['name','code','close'])
                # 返回的dataframe中的全都是md1>md2的，则如果股票代码不在其中就可以卖出
                if not result[result['code'].str.contains(code)].empty:
                    return_info = bigdata1.sell(code,amount,now)
                    # 已经存在下单单号
                    if 'entrust_no' in return_info.keys():
                        update_dict[return_info['entrust_no']] = id
# 根据当日成交来判断update_info中的信息是否应该更新
print("here")
trade_info = pd.DataFrame(bigdata1.today_trades)
del_key = []
for key,value in update_dict.items():
    res = trade_info[trade_info['委托序号']==key]
    if res.empty:
        continue
    # 当前记录已经成交
    else:
        del_key.append(key)
        state = 0
        price = res['成交价格']
        updated_time = res['成交时间']
        updated_time = datetime.datetime.strptime(updated_time,"%Y-%m-%d")
        updated_time = datetime.date(updated_time)
        if res['买卖标志'] == '买入':
            state = 1
            sql = 'update stockorder set state =\'%d\',updated_time = \'%s\', buy_price = \'%f\' where id = \'%d\'' %(state,updated_time,price,id)
            cur.execute(sql)
        else:
            state = 2
            sql = 'update stockorder set state =\'%d\',updated_time = \'%s\', sell_price = \'%f\' where id = \'%d\'' %(state,updated_time,price,id)
            cur.execute(sql)
print("there")
# 删去已更新的信息
for element in del_key:
    del update_dict[element]

002605.XSHE 26.88
600990.XSHG 22.31
here
there


In [30]:
sql = 'select id,code,state,updated_time,stra_param_buy,stra_param_sell,amount from stockorder where state = 0 or state = 1'
cur.execute(sql)
order = cur.fetchall()
order_df = pd.DataFrame(order, columns=['id','code','state','updated_time','stra_param_buy','stra_param_sell','amount'])
for index,row in order_df.iterrows():
    code = row['code']
    id = row['id']
    print(type(row['stra_param_sell']))
    amount = row['amount']
    code_denomalized = denormalize_code(code)
    # 如果传入数据没有这个股票的实时信息，则略过
    if code_denomalized not in data.keys():
        continue
    else:
        # 捕获当前的价格
        now = data[code_denomalized]['now']
        print(code,now)
        # 如果状态为未买入
        if row['state'] == 0:
            stra_buy = re.split(r"[ ,]",row['stra_param_buy'])
            print("buy",stra_buy)
            if stra_buy[0] == 'price:':
                # 如果是price策略，则判断
                target_price = float(stra_buy[1])
                if now <= target_price:
                    # 下单会返回一个委托单号{'entrust_no':},如果成交/错误则返回{'message':}
                    return_info = bigdata1.buy(code,amount,target_price)
                    # 已经存在下单单号
                    if 'entrust_no' in return_info.keys():
                        update_dict[return_info['entrust_no']] = id
            elif stra_buy[0] == 'percent:':
                # 如果是percent策略，则判断买入价格是否到达对应百分比，默认买入是1-percent
                target_percent = float(stra_buy[1])
                target_price = float(stra_buy[3])
                target_price = target_price*(1-0.01*target_percent)
                if now <= target_price:
                    # 下单会返回一个委托单号{'entrust_no':},如果成交/错误则返回{'message':}
                    return_info = bigdata1.buy(code,amount,target_price)
                    # 已经存在下单单号
                    if 'entrust_no' in return_info.keys():
                        update_dict[return_info['entrust_no']] = id
            elif stra_buy[0] == 'ma1:':
                # 如果是MA策略，则判断对应股票是否在redis中
                current_day = datetime.date.today()
                DateKey = current_day.strftime("%Y%m%d")+":"+str(md1)+str(md2)
                result = r.get(DateKey)
                if result == None:
                    md1 = int(stra_buy[1])
                    md2 = int(stra_buy[3])
                    result = get_ma(md1=md1,md2=md2)

                result = json.loads(result)
                result = pd.DataFrame(result,columns=['name','code','close'])
                # 返回的dataframe中的全都是md1>md2的，则如果股票代码在其中就可以买入
                if not result[result['code'].str.contains(code)].empty:
                    return_info = bigdata1.buy(code,amount,now)
                    # 已经存在下单单号
                    if 'entrust_no' in return_info.keys():
                        update_dict[return_info['entrust_no']] = id

        if row['state'] == 1:
            # 如果状态为已买入,需要检查一下是否当天买入
            if row['updated_time'] == datetime.datetime.today:
                continue
            stra_sell = re.split(r"[ ,]",row['stra_param_sell'])
            print("sell",stra_sell)
            if stra_sell[0] == 'price:':
                # 如果是price策略，则判断
                target_price = float(stra_sell[1])
                if now >= target_price:
                    # 下单会返回一个委托单号{'entrust_no':},如果成交/错误则返回{'message':}
                    return_info = bigdata1.sell(code,amount,target_price)
                    # 已经存在下单单号
                    if 'entrust_no' in return_info.keys():
                        update_dict[return_info['entrust_no']] = id
            elif stra_sell[0] == 'percent:':
                # 如果是percent策略，则判断
                target_percent = float(stra_sell[1])
                target_price = float(stra_sell[3])
                target_price = target_price*(1+0.01*target_percent)
                if now >= target_price:
                    # 下单会返回一个委托单号{'entrust_no':},如果成交/错误则返回{'message':}
                    return_info = bigdata1.sell(code,amount,target_price)
                    # 已经存在下单单号
                    if 'entrust_no' in return_info.keys():
                        update_dict[return_info['entrust_no']] = id
            elif stra_sell[0] == 'ma1:':
                # 如果是MA策略，则判断
                # 如果是MA策略，则判断对应股票是否在redis中
                current_day = datetime.date.today()
                DateKey = current_day.strftime("%Y%m%d")+":"+str(md1)+str(md2)
                result = r.get(DateKey)
                if result == None:
                    md1 = int(stra_sell[1])
                    md2 = int(stra_sell[3])
                    result = get_ma(md1=md1,md2=md2)


<class 'str'>
002605.XSHE 26.88
buy ['price:26']
<class 'str'>
600990.XSHG 22.31
buy ['precent:2.3%', 'price:22.82']
<class 'str'>
600682.XSHG 7.9
sell ['ma1:5', 'ma2:10', 'mode:day']
<class 'str'>
002270.XSHE 12.27
sell ['ma1:5', 'ma2:20', 'mode:day']
<class 'str'>
600989.XSHG 14.37
buy ['price:15.6']


In [9]:
current = datetime.datetime.now()
before = current + datetime.timedelta(days=-90)
current = current.strftime("%Y%m%d")
before = before.strftime("%Y%m%d")
trade_cal = pro.trade_cal(
    start_date= before, end_date=current, is_open='1'
)['cal_date'].tolist()

In [17]:
before = trade_cal[10]
yesterday = trade_cal[1]
sql = 'select date,name,code,close from daily_stock where date >= \'%s\' and date<= \'%s\''%(before,yesterday)
cur.execute(sql)
data = cur.fetchall()
data = pd.DataFrame(data,columns=['date','name','code','close'])
data['name'] = data['name'].str.strip()
data['code'] = data['code'].str.strip()
data

,date,name,code,close
0,2023-09-01,明泰铝业,601677.XSHG,14.680
1,2023-09-01,壹网壹创,300792.XSHE,25.280
2,2023-09-01,佳禾智能,300793.XSHE,17.660
3,2023-09-01,米奥会展,300795.XSHE,36.400
4,2023-09-01,贝斯美,300796.XSHE,12.800
5,2023-09-01,钢研纳克,300797.XSHE,11.570
6,2023-09-01,锦鸡股份,300798.XSHE,6.960
7,2023-09-01,*ST左江,300799.XSHE,284.980
8,2023-09-01,力合科技,300800.XSHE,13.070
9,2023-09-01,泰和科技,300801.XSHE,16.270


In [ ]:
# 从数据库中返回某个股票的ma1和ma2均线值,默认ma1<ma2
def get_ma(ma1:int, ma2:int):
    # 给出两个日期限制
    before = trade_cal[ma2]
    yesterday = trade_cal[1]
    sql = 'select date,name,code,close from daily_stock where date >= \'%s\' and date<= \'%s\''%(before,yesterday)
    cur.execute(sql)
    data = cur.fetchall()
    data = pd.DataFrame(data,columns=['date','name','code','close'])
    data['name'] = data['name'].str.strip()
    data['code'] = data['code'].str.strip()
    return data
data = get_ma(1,5)
df = pd.DataFrame(columns=['name','code','close'])
day_before = trade_cal[1]
sql1 = 'SELECT name,close,code FROM daily_stock where date = \'%s\'' %(day_before)
cur.execute(sql1)
val = cur.fetchall()
df = pd.DataFrame(val,columns=['name','close','code'])
code_list = [df.loc[i]['code'].split(' ')[0] for i in range(len(df))]
# val = data[data['date']<datetime.date(*map(int,"2023-09-01".split('-')))]
for code in code_list:
    info = data[data['code']==code]
    new_info = info.sort_values(by='date',inplace=False,ascending=False)
    val_ma1 = new_info.iloc[0:5,3].mean()
    val_ma2 = new_info.iloc[:,3].mean()
    if val_ma1 > val_ma2:
        pd.DataFrame(np.insert(df.values,0,values=[name,close,code],axis=0))
df